In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rps = pd.read_csv('/kaggle/input/check-cv-results/rps.csv')


In [ ]:
rps[rps['std'].notna()].sort_values('max')

In [ ]:
candidates = rps[
    (rps['max']<.1598)
    &(rps['std'].notna())
].copy()

In [ ]:
candidates

In [ ]:
ir = pd.read_csv('/kaggle/input/check-cv-results/ir.csv')
ir = ir.set_index(['f', 'port', 'scaling'])
for col in ir.columns:
    candidates[col] = candidates.set_index(['f', 'port', 'scaling']).index.map(ir[col])

candidates = candidates[candidates[ir.columns].notna().all(axis=1)].copy().reset_index(drop=True)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(candidates[ir.columns])
colours = ['blue', 'red', 'green']
for i, (key, val) in enumerate(candidates[ir.columns].mean().to_dict().items()):
    fig.add_shape(type='line',
                x0=0,
                y0=val,
                x1=len(candidates),
                y1=val,
                line=dict(color=colours[i],),
                xref='x',
                yref='y'
)
fig

In [ ]:
naive_baseline = pd.read_csv('/kaggle/input/check-naive-cv/ir_results.csv').iloc[:, 0]
naive_baseline

In [ ]:
final_candidates = candidates[
    (candidates[ir.columns[0]]>=naive_baseline.iloc[0])
    &(candidates[ir.columns[1]]>=naive_baseline.iloc[1])
    &(candidates[ir.columns[2]]>=naive_baseline.iloc[2])
]
final_candidates

In [ ]:
# ok, these are all the ones we'll use

In [ ]:
final_candidates['f'].tolist()

In [ ]:
final_candidates['port'].tolist()

In [ ]:
final_candidates['scaling'].tolist()

In [ ]:
final_candidates.to_csv('final_candidates.csv', index=False)